In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as RMSE


train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

display(train_df.head())
print(train_df.select_dtypes('object').columns)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


Index(['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title'],
      dtype='object')


In [7]:
train_dummies = pd.get_dummies(train_df[['brand', 'fuel_type', 'clean_title']])
train_dummies

train_data = train_df.select_dtypes(exclude=['object'])
scaler = MinMaxScaler()
train_data = pd.concat([train_data, train_dummies], axis = 1)
train_data.head()

,id,model_year,milage,price,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,...,brand_Volkswagen,brand_Volvo,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,clean_title_Yes
0,0,2018,74349,11000,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,1,2007,80000,8250,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
2,2,2009,91491,15000,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
3,3,2022,2437,63500,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
4,4,2001,111000,7850,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [11]:
X = train_data.drop(['price'], axis = 1).values
X = scaler.fit_transform(X)

y = train_data['price'].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 5)

gbt = GradientBoostingRegressor(n_estimators = 300, max_depth = 1, random_state = 5)
gbt.fit(X_train, y_train)

y_pred = gbt.predict(X_test)

rmse_test = RMSE(y_test, y_pred)
rmse_test

75583.423608638

In [10]:
test_dummies = pd.get_dummies(test_df[['brand', 'fuel_type', 'clean_title']])
test_data = test_df.select_dtypes(exclude=['object'])

test_data = pd.concat([test_data, test_dummies], axis = 1)
test_data.head()

test_X = scaler.fit_transform(test_data.values)
test_pred = gbt.predict(test_X)
test_pred


array([59548.54212778, 21731.79273734, 72541.5892338 , ...,
       16899.05491241, 45820.43036411, 14753.70150794])

In [114]:
submission = pd.DataFrame({'id':test_df.id, 'price': test_pred})
submission.to_csv('submission.csv', index=False)